<a href="https://colab.research.google.com/github/Maoelan/zero-shot-news-classification/blob/main/Zero_Shot_Classification_for_News_Articles_Using_BART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import huggingface_hub
huggingface_hub.login(token='')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

In [3]:
model_name = 'facebook/bart-large-mnli'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [4]:
zero_shot_classifier = pipeline('zero-shot-classification', model=model, tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [5]:
def zero_shot_classify(texts, candidate_labels):
    results = []
    for text in texts:
        result = zero_shot_classifier(text, candidate_labels)
        results.append(result)
    return results

In [6]:
def get_user_input_multiple():
    print("Masukkan teks untuk klasifikasi (ketik 'selesai' untuk berhenti):")
    texts = []
    while True:
        text = input("Teks: ")
        if text.lower() == 'selesai':
            break
        texts.append(text)

    labels = input("Masukkan label kandidat dipisahkan dengan koma: ")
    candidate_labels = [label.strip() for label in labels.split(',')]

    return texts, candidate_labels

In [7]:
import pandas as pd

def print_results(results):
    data = []
    for result in results:
        text = result['sequence']
        labels = result['labels']
        scores = result['scores']
        # Ambil label dengan skor tertinggi untuk setiap teks
        best_label = labels[0]
        best_score = scores[0]
        for label, score in zip(labels, scores):
            if score > best_score:
                best_label = label
                best_score = score
        data.append({
            'Text': text,
            'Label': best_label,
            'Score': best_score
        })

    df = pd.DataFrame(data)
    print(df)

    filename = 'classification_results.csv'
    df.to_csv(filename, index=False)
    print(f"Results saved to {filename}")

In [14]:
texts, candidate_labels = get_user_input_multiple()

Masukkan teks untuk klasifikasi (ketik 'selesai' untuk berhenti):
Teks: Major computer outage could impact 911 services but ‘is not a cyberattack’
Teks: American passenger's flight from Japan cancelled, lounge is full and baggage delayed
Teks: Chaos for retail and banking worldwide
Teks: Trump accepts nomination, promising a 'thrilling chapter' for U.S. in rally-like speech
Teks: Trump describes in detail being shot by would-be assassin
Teks: Fact-checking Trump's RNC speech on taxes, crime, foreign policy and more
Teks: Four dead in a week: Heat leads to tragedies in state and national parks
Teks: More than 300 suspected heat-related deaths under investigation in Phoenix area
Teks: Will Team USA cruise to gold in track? Not if the Jamaicans have anything to say about it
Teks: College professor and State Department's 'Ambassador of Skateboarding' is heading to Paris
Teks: ‘I couldn’t go anywhere without getting mobbed’: Gabby Douglas and Nastia Liukin recall joining gymnastics royalty


In [15]:
results = zero_shot_classify(texts, candidate_labels)
print_results(results)

                                                 Text          Label     Score
0   Major computer outage could impact 911 service...     Technology  0.342973
1   American passenger's flight from Japan cancell...         Social  0.313151
2              Chaos for retail and banking worldwide        Economy  0.397684
3   Trump accepts nomination, promising a 'thrilli...       Politics  0.478528
4   Trump describes in detail being shot by would-...       Politics  0.370109
5   Fact-checking Trump's RNC speech on taxes, cri...       Politics  0.709554
6   Four dead in a week: Heat leads to tragedies i...    Environment  0.306325
7   More than 300 suspected heat-related deaths un...         Health  0.465909
8   Will Team USA cruise to gold in track? Not if ...         Sports  0.890346
9   College professor and State Department's 'Amba...         Sports  0.794523
10  ‘I couldn’t go anywhere without getting mobbed...         Social  0.345432
11  Thomas Crooks appears to have posted ominous m..